In [25]:
from os import listdir
import cv2


### Function to get label by video id

In [27]:
import sys
from os import path

module_path = path.abspath(path.join("../utils"))
if module_path not in sys.path:
    sys.path.append(f"{module_path}/ResNets20")

from data_reader import get_label_from_id


### Input folder and img names

In [26]:
input_folder = "../../data/WLASL2000_frames"
input_img_names = listdir(input_folder)


### Divide input imgs in train, validation and test

In [ ]:
train, test, validation = 0.7, 0.1, 0.2


In [29]:
image = cv2.imread(f"{input_folder}/{input_img_names[1]}")
cv2.imshow("image", image)
cv2.waitKey(0)


-1